In [1]:
import pandas as pd
import pymysql

In [2]:
db_name = "spain_train_system"
db_host = "localhost"
db_username = "root"
db_password = "Garracrema25"

In [3]:
try:
    conn = pymysql.connect(host=db_host,
                            port=int(3306),
                            user='root',
                            passwd=db_password,
                            db=db_name)
except e:
    print(e)

In [4]:
if conn:
    print("Connection Successful")
else:
    print("There was an error")

Connection Successful


In [5]:
trips_info = pd.read_sql_query("SELECT * FROM trips_info", conn)
trips_info

,Unnamed: 0,insert_date,origin,destination,start_date,end_date,train_type,price,train_class,fare
0,702,2019-04-19 05:37:35,PONFERRADA,MADRID,2019-06-02 15:00:00,2019-06-02 19:42:00,MD-AVE,59.50,Turista con enlace,Flexible
1,703,2019-04-19 05:37:35,PONFERRADA,MADRID,2019-06-02 17:15:00,2019-06-02 23:03:00,MD-AVE,34.65,Turista con enlace,Promo +
2,704,2019-04-19 05:37:35,PONFERRADA,MADRID,2019-06-02 17:15:00,2019-06-02 23:10:00,MD-LD,39.95,Turista con enlace,Promo +
3,705,2019-04-19 05:37:35,PONFERRADA,MADRID,2019-06-02 17:15:00,2019-06-02 22:14:00,MD-AVE,40.60,Turista con enlace,Promo +
4,706,2019-04-19 05:37:35,PONFERRADA,MADRID,2019-06-02 18:55:00,2019-06-02 23:03:00,ALVIA,27.90,Turista,Promo
...,...,...,...,...,...,...,...,...,...,...
184238,2217553,2019-04-16 12:28:25,SEVILLA,MADRID,2019-06-02 18:45:00,2019-06-02 21:17:00,AVE,76.30,Turista,Flexible
184239,2217554,2019-04-16 12:28:25,SEVILLA,MADRID,2019-06-02 19:15:00,2019-06-02 21:36:00,AVE,76.30,Turista,Flexible
184240,2217555,2019-04-16 12:28:25,SEVILLA,MADRID,2019-06-02 19:19:00,2019-06-02 21:59:00,INTERCITY,62.20,Turista,Flexible
184241,2217556,2019-04-16 12:28:25,SEVILLA,MADRID,2019-06-02 19:45:00,2019-06-02 22:17:00,AVE,64.05,Turista Plus,Promo


Analyze values for each dimension: origin, destination, train type, price, train class, fare 

In [6]:
n_of_origin = pd.read_sql_query("""
SELECT 
COUNT(DISTINCT origin) AS count_of_origins
FROM trips_info
""", conn)

n_of_origin

,count_of_origins
0,5


In [7]:
n_of_dest = pd.read_sql_query("""
SELECT 
COUNT(DISTINCT destination) AS count_of_destinations
FROM trips_info
""", conn)

n_of_dest

,count_of_destinations
0,5


In [8]:
n_of_routes = pd.read_sql_query("""
SELECT
DISTINCT concat(origin,'-',destination) AS route
FROM trips_info
""", conn)

n_of_routes

,route
0,PONFERRADA-MADRID
1,MADRID-VALENCIA
2,MADRID-BARCELONA
3,SEVILLA-MADRID
4,MADRID-PONFERRADA
5,VALENCIA-MADRID
6,BARCELONA-MADRID
7,MADRID-SEVILLA


In [9]:
train_types = pd.read_sql_query("""
SELECT
DISTINCT train_type
FROM trips_info
""", conn)

train_types

,train_type
0,MD-AVE
1,MD-LD
2,ALVIA
3,REGIONAL
4,AVE
5,INTERCITY
6,AVE-MD
7,AVE-LD
8,R. EXPRES
9,AVE-TGV


In [10]:
train_class = pd.read_sql_query("""
SELECT
DISTINCT train_class
FROM trips_info
""", conn)

train_class

,train_class
0,Turista con enlace
1,Turista
2,Preferente
3,Turista Plus
4,Cama Turista


In [11]:
fares = pd.read_sql_query("""
SELECT
DISTINCT fare
FROM trips_info
""", conn)

fares

,fare
0,Flexible
1,Promo +
2,Promo
3,Adulto ida
4,Mesa


## Which are the most transitted routes?

In [12]:
trips_per_route = pd.read_sql_query("""
SELECT
DISTINCT concat(origin,'-',destination) AS route, COUNT(fare) AS count_of_trips
FROM trips_info
GROUP BY route
ORDER BY count_of_trips DESC
""", conn)

trips_per_route

,route,count_of_trips
0,MADRID-BARCELONA,36528
1,BARCELONA-MADRID,35845
2,MADRID-VALENCIA,33491
3,VALENCIA-MADRID,31860
4,MADRID-SEVILLA,21208
5,SEVILLA-MADRID,20930
6,MADRID-PONFERRADA,2205
7,PONFERRADA-MADRID,2176


## Which routes has the most variety of train types? and train classes?

In [13]:
train_types_per_route = pd.read_sql_query("""
SELECT
DISTINCT concat(origin,'-',destination) AS route, COUNT(DISTINCT train_type) AS count_train_types
FROM trips_info
GROUP BY route
ORDER BY count_train_types DESC
""", conn)

train_types_per_route

,route,count_train_types
0,MADRID-PONFERRADA,6
1,SEVILLA-MADRID,6
2,MADRID-SEVILLA,5
3,MADRID-VALENCIA,5
4,PONFERRADA-MADRID,4
5,VALENCIA-MADRID,4
6,BARCELONA-MADRID,3
7,MADRID-BARCELONA,3


In [14]:
train_classes_per_route = pd.read_sql_query("""
SELECT
DISTINCT concat(origin,'-',destination) AS route, COUNT(DISTINCT train_class) AS count_train_classes
FROM trips_info
GROUP BY route
ORDER BY count_train_classes DESC
""", conn)

train_classes_per_route

,route,count_train_classes
0,MADRID-SEVILLA,4
1,MADRID-VALENCIA,4
2,SEVILLA-MADRID,4
3,VALENCIA-MADRID,4
4,BARCELONA-MADRID,3
5,MADRID-BARCELONA,3
6,MADRID-PONFERRADA,3
7,PONFERRADA-MADRID,2


## Which route has the most offers in terms of hours?

In [15]:
hours_per_route = pd.read_sql_query("""
WITH hours AS
(
SELECT
DISTINCT concat(origin,'-',destination) AS route,
concat(HOUR(start_date),":",MINUTE(start_date)) AS time
FROM trips_info
)
SELECT route, COUNT(DISTINCT time) AS available_daily_trips
FROM hours 
GROUP BY route
ORDER BY available_daily_trips DESC
""", conn)

hours_per_route

,route,available_daily_trips
0,BARCELONA-MADRID,32
1,MADRID-BARCELONA,32
2,SEVILLA-MADRID,27
3,MADRID-SEVILLA,26
4,MADRID-VALENCIA,23
5,VALENCIA-MADRID,18
6,PONFERRADA-MADRID,4
7,MADRID-PONFERRADA,3


## Which route has the most offers in terms of price range?

In [16]:
prices_per_route = pd.read_sql_query("""
SELECT
DISTINCT concat(origin,'-',destination) AS route, MAX(price) AS priciest_ticket, MIN(price) AS cheapest_ticket
FROM trips_info
GROUP BY route
""", conn)

prices_per_route

,route,priciest_ticket,cheapest_ticket
0,PONFERRADA-MADRID,59.5,27.9
1,MADRID-VALENCIA,73.1,16.6
2,MADRID-BARCELONA,206.8,32.3
3,SEVILLA-MADRID,128.5,22.9
4,MADRID-PONFERRADA,74.2,31.2
5,VALENCIA-MADRID,73.1,16.6
6,BARCELONA-MADRID,129.3,32.3
7,MADRID-SEVILLA,87.4,22.9


In [17]:
count_prices_per_route = pd.read_sql_query("""
SELECT
DISTINCT concat(origin,'-',destination) AS route, COUNT(DISTINCT price) AS count_of_different_prices
FROM trips_info
GROUP BY route
ORDER BY count_of_different_prices DESC
""", conn)

count_prices_per_route

,route,count_of_different_prices
0,BARCELONA-MADRID,51
1,MADRID-BARCELONA,47
2,SEVILLA-MADRID,37
3,MADRID-SEVILLA,33
4,VALENCIA-MADRID,25
5,MADRID-VALENCIA,22
6,MADRID-PONFERRADA,13
7,PONFERRADA-MADRID,8


## Which are the most used fares by route?

In [18]:
count_fares_per_route = pd.read_sql_query("""
SELECT
DISTINCT concat(origin,'-',destination) AS route, fare, COUNT(fare) AS count_of_fares
FROM trips_info
GROUP BY route, fare
ORDER BY route
""", conn)

count_fares_per_route

,route,fare,count_of_fares
0,BARCELONA-MADRID,Adulto ida,3167
1,BARCELONA-MADRID,Flexible,2371
2,BARCELONA-MADRID,Promo,30307
3,MADRID-BARCELONA,Adulto ida,2259
4,MADRID-BARCELONA,Flexible,1382
5,MADRID-BARCELONA,Mesa,66
6,MADRID-BARCELONA,Promo,32821
7,MADRID-PONFERRADA,Flexible,905
8,MADRID-PONFERRADA,Promo,676
9,MADRID-PONFERRADA,Promo +,624


## Which are the cheapest and priciest hours to travel (by route)? 

In [35]:
cheapest_hour_per_route = pd.read_sql_query("""
SELECT
concat(origin,'-',destination) AS route, concat(HOUR(start_date),":",MINUTE(start_date)) AS time, price
FROM trips_info
""", conn)

cheapest_hour_per_route

,route,time,price
0,PONFERRADA-MADRID,15:0,59.50
1,PONFERRADA-MADRID,17:15,34.65
2,PONFERRADA-MADRID,17:15,39.95
3,PONFERRADA-MADRID,17:15,40.60
4,PONFERRADA-MADRID,18:55,27.90
...,...,...,...
184238,SEVILLA-MADRID,18:45,76.30
184239,SEVILLA-MADRID,19:15,76.30
184240,SEVILLA-MADRID,19:19,62.20
184241,SEVILLA-MADRID,19:45,64.05
